## Mobility Robustness Optimization (MRO)

Takes in new observation data to train or update the bayesian digital twin models. It processes the input data and updates the model to better reflect the current network conditions.

Then MRO optimizes the mobility robustness by solving the underlying problem using the trained model: finding optimal `HYST` and `TTT`. There are two solve approaches shown: 

- Simple MRO
- Reinforced MRO

In [ ]:
import sys
from pathlib import Path
sys.path.append(f"{Path().absolute().parent}")

In [ ]:
import pandas as pd
import numpy as np
from apps.mobility_robustness_optimization.simple_mro import SimpleMRO
from apps.mobility_robustness_optimization.mro_rl import ReinforcedMRO

*unzip the `data/mro_data.zip` file to get `data/mro_data/` folder*

# Showcasing **Simple MRO** Solving Approach

use the following example `topology` and `mobility_model_params` to initiate MRO.

In [ ]:
topology = pd.read_csv('data/mro_data/mro_topology.csv')
topology

In [ ]:
mobility_model_params = {
    "ue_tracks_generation": {
            "params": {
                "simulation_duration": 3600,
                "simulation_time_interval_seconds": 0.01,
                "num_ticks": 50,
                "num_batches": 1,
                "ue_class_distribution": {
                    "stationary": {
                        "count": 10,
                        "velocity": 0,
                        "velocity_variance": 1
                    },
                    "pedestrian": {
                        "count": 5,
                        "velocity": 2,
                        "velocity_variance": 1
                    },
                    "cyclist": {
                        "count": 5,
                        "velocity": 5,
                        "velocity_variance": 1
                    },
                    "car": {
                        "count": 12,
                        "velocity": 20,
                        "velocity_variance": 1
                    }
                },
                "lat_lon_boundaries": {
                    "min_lat": -90,
                    "max_lat": 90,
                    "min_lon": -180,
                    "max_lon": 180
                },
                "gauss_markov_params": {
                    "alpha": 0.5,
                    "variance": 0.8,
                    "rng_seed": 42,
                    "lon_x_dims": 100,
                    "lon_y_dims": 100,
                    "// TODO": "Account for supporting the user choosing the anchor_loc and cov_around_anchor.",
                    "// Current implementation": "the UE Tracks generator will not be using these values.",
                    "// anchor_loc": {},
                    "// cov_around_anchor": {}
            }
        }
    }
}

<b>Optionally,</b> use mobility model to get `alpha` of your data and set it to params.

In [ ]:
# [OPTIONAL] run this cell to get alpha calculated from the data into mobility_model_params

from radp.digital_twin.mobility.param_regression import get_predicted_alpha

# 20 UEs x 50 ticks = 1000 rows
ue_data = pd.read_csv("data/mro_data/UE_data_20UE_100ticks.csv") # change this to the data you want to use
ue_data = ue_data.rename(columns={'latitude': 'lat', 'longitude': 'lon'})

# set random initial alpha
alpha0 = np.random.choice(np.arange(0, 1.1, 0.1))

alpha = get_predicted_alpha(ue_data, alpha0 = alpha0, seed = 42)

print(f"Learned alpha: {alpha:.2f}\n")

mobility_model_params["ue_tracks_generation"]["params"]["gauss_markov_params"]["alpha"] = alpha

In [ ]:
mro = SimpleMRO(mobility_model_params, topology)

In [ ]:
# initially bayesian_digital_twins is empty
print(f"bayesian_digital_twins: {mro.bayesian_digital_twins}")

- prepare `new_data` for training/updating `bayesian_digital_twins`

    - `new_data` should have received power data in cartesian df format. required cols ['latitude', 'longitude', 'cell_id', 'cell_rxpwr_dbm']

In [ ]:
# 20 UEs x 100 ticks x 3 cells cartesian = 6000 rows
ue_data_with_rxpower = pd.read_csv("data/mro_data/UE_data_with_rxpower_20UE_100ticks_train.csv") # change this to the data you want to use
input_data = ue_data_with_rxpower.copy()

input_data.head()

In [ ]:
# train bayesian_digital_twins from scratch
mro.train_or_update_rf_twins(new_data=input_data)

In [ ]:
mro.bayesian_digital_twins # bayesian_digital_twins is trained for each cell_id

can save trained/updated `bayesian_digital_twins`

In [ ]:
saving_dir_relative_path = "data/mro_data/"

mro.save_bdt(saving_dir_relative_path) # True indicates save is successful

call `solve()` method to get optimized `HYST` and `TTT`

In [ ]:
# adjust n_epochs for better performance
hyst, ttt = mro.solve(n_epochs=100)

In [ ]:
from notebooks.radp_library import mro_plot_scatter, plot_sinr_db_by_ue, mro_score_3d_plot
from radp.digital_twin.utils.constants import RLF_THRESHOLD
from radp.digital_twin.utils.cell_selection import perform_attachment_hyst_ttt

attached_df = perform_attachment_hyst_ttt(mro.simulation_data, hyst, ttt, RLF_THRESHOLD)
mro_plot_scatter(attached_df, topology)

In [ ]:
mro_score_3d_plot(mro.score)

In [ ]:
ue_id = 0 # change this to the UE you want to plot
plot_sinr_db_by_ue(attached_df, mro.simulation_data, ue_id)

can load this `bayesian_digital_twins` later when needed

In [ ]:
mro.bayesian_digital_twins = {} # bayesian_digital_twins is empty again

pkl_file_path = "data/mro_data/digital_twins.pkl"
mro.load_bdt(pkl_file_path) # True indicates load is successful

In [ ]:
# Dummy solve call to avoid fantasy observation error: ensuring all test independent caches exist
mro.solve(n_epochs=2)

let's try updating the `bayesian_digital_twins` with new observations

In [ ]:
# 20 UEs x 100 ticks x 3 cells cartesian = 6000 rows
new_obeservations = pd.read_csv("data/mro_data/UE_data_with_rxpower_20UE_100ticks_update.csv") # change this to the data you want to use
input_data = new_obeservations.copy()

input_data.head()

In [ ]:
# update bdt with new observations, calling train_or_update_rf_twin() again
mro.train_or_update_rf_twins(input_data)

can solve with updated `bayesian_digital_twins`

In [ ]:
# adjust n_epochs for better performance
mro.solve(n_epochs=100)

# Showcasing **Reinforced MRO** Solving Approach

use the following example `topology` and `mobility_model_params` to initiate MRO.

In [ ]:
rl_mro = ReinforcedMRO(mobility_model_params, topology)

In [ ]:
print(f"bayesian_digital_twins: {rl_mro.bayesian_digital_twins}", end='\n\n') # bayesian_digital_twins is empty initially

pkl_file_path = "data/mro_data/digital_twins.pkl" # run previous section to have this file
rl_mro.load_bdt(pkl_file_path) # True indicates load is successful

In [ ]:
rl_mro.bayesian_digital_twins

- load `new_data` for training/updating `bayesian_digital_twins`

    - `new_data` should have received power data in cartesian df format. required cols ['latitude', 'longitude', 'cell_id', 'cell_rxpwr_dbm']

In [ ]:
# 20 UEs x 50 ticks x 3 cells cartesian = 3000 rows
ue_data_with_rxpower = pd.read_csv("data/mro_data/UE_data_with_rxpower_20UE_50ticks.csv")
input_data = ue_data_with_rxpower.copy()

input_data

In [ ]:
# Dummy solve call to avoid fantasy observation error: ensuring all test independent caches exist
rl_mro.solve(total_timesteps=2)

In [ ]:
# update bdt with new data
rl_mro.train_or_update_rf_twins(input_data)

Solve using updated `bayesian_digital_twins`

In [ ]:
# adjust total_timesteps for better performance
hyst, ttt = rl_mro.solve(total_timesteps=100)

In [ ]:
attached_df = perform_attachment_hyst_ttt(mro.simulation_data, hyst, ttt, RLF_THRESHOLD)
mro_plot_scatter(attached_df, topology)

In [ ]:
ue_id = 0 # change this to the UE you want to plot
plot_sinr_db_by_ue(attached_df, mro.simulation_data, ue_id)